In [ ]:
!pip install mauve-text

In [ ]:
!pip install transformers

In [ ]:
# Code to mount the drive 
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/685/catr/'

/content/drive/.shortcut-targets-by-id/12c1zkm0_oa8VcOfsUa8Tn_YcYdyPpSlP/685/catr


In [ ]:
from transformers import GPT2Tokenizer, GPT2Model
import pandas as pd
import numpy as np
import mauve

In [ ]:
emogen_prediction_test_data = pd.read_csv("/content/drive/MyDrive/685/catr/IntermediateFineTuning/checkpoint_base_emogen.csv")
emogen_baseline_prediction_test_data = pd.read_csv("/content/drive/MyDrive/685/catr/IntermediateFineTuning/BaselineEmogenTestSetResults.csv")
emogen_actual_prediction_test_data = pd.read_csv("/content/drive/MyDrive/685/emogen/annotations/test_caption.csv")
# test_data = pd.read_csv("/content/drive/MyDrive/685/catr/IntermediateFineTuning/checkpoint_base_emogen.csv")
# val_data = pd.read_csv("/content/drive/MyDrive/685/sarcasm/val_data/caption/val_caption.csv")
# ptext = val_data['comment'].tolist()[:40]
# qtext = test_data['comment'].tolist()[:40]

In [ ]:
emogen_prediction_test_data = pd.read_csv("/content/drive/MyDrive/685/catr/IntermediateFineTuning/checkpoint_base_emogen.csv")
emogen_baseline_prediction_test_data = pd.read_csv("/content/drive/MyDrive/685/catr/IntermediateFineTuning/BaselineEmogenTestSetResults.csv")
emogen_actual_prediction_test_data = pd.read_csv("/content/drive/MyDrive/685/emogen/annotations/test_caption.csv")

In [ ]:
ptext = emogen_prediction_test_data['caption'].tolist()[:80]
qtext = emogen_baseline_prediction_test_data['caption'].tolist()[:80]
rtext = emogen_actual_prediction_test_data['comment'].tolist()[:80]

In [ ]:
len(emogen_prediction_test_data['caption'].tolist()), len(emogen_baseline_prediction_test_data['caption'].tolist()),len(emogen_actual_prediction_test_data['comment'].tolist())

(101, 465, 476)

In [ ]:
def calculate_muave(ptext, qtext):
  """
  Function to calculate final mauve metrics
  """
  tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
  model = GPT2Model.from_pretrained('gpt2')
  tokenizer.pad_token=tokenizer.eos_token
  pta = tokenizer(ptext, return_tensors='pt',padding=True)
  qta = tokenizer(qtext,return_tensors='pt',padding=True)
  outputs = model(**pta)
  last_hidden_states_1 = outputs.last_hidden_state
  outputs2 = model(**qta)
  last_hidden_states_2 = outputs2.last_hidden_state
  pta_feats = last_hidden_states_1[:,0,:].detach().numpy()
  qta_feats = last_hidden_states_2[:,0,:].detach().numpy()

  metrics = mauve.compute_mauve(p_features=pta_feats,q_features=qta_feats)

  return metrics

In [ ]:
metrics = calculate_muave(ptext,rtext)
metrics.mauve, metrics.frontier_integral

(0.06394696334967412, 0.4295959031596762)

In [ ]:
metrics2 = calculate_muave(qtext,rtext)
metrics2.mauve, metrics2.frontier_integral

(0.13753984274824554, 0.4299183931586769)

0.13753984274824554

0.4299183931586769